In [79]:
import time
import sched
from datetime import datetime, tzinfo
from elasticsearch import Elasticsearch
#matplotlib.rc('font', **{'size': 15})
from elasticsearch.helpers import scan




In [80]:
#define scheduler
s = sched.scheduler(time.time, time.sleep)

#define variables
interval = 60*10 #update every [interval] seconds

verbose = True

pattern = '%Y-%m-%d %H:%M:%S'



#debug
if verbose: print('Local time Chicago: ',datetime.now().strftime(pattern))

#number of successful transfers in interval
transfers = 0
#number of failed transfers
failures = 0

Local time Chicago:  2017-04-25 03:47:03


In [81]:
def process_last():
    print()
    #reset counters
    transfers = 0
    failures = 0
    now_datetime = datetime.now().strftime(pattern)
    if verbose: print('The clock is: ',now_datetime)
    if verbose: print('Epoch time: ',int(time.time())*1000)

    now = int(time.time()) * 1000
    
    past = now - interval*1000 #convert interval to epoch milliseconds
   
    
    
    query_done = {
        "size": 0,
        "query": {
        "bool": {
        "must": [
        {
            "range": {
            "@timestamp": {
                "gte": past,
                "lte": now,
                "format": "epoch_millis"
                }
            }
        },
            {"term": {"event_type": "transfer-done"}},
            {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
        }
    }
    query_failed = {
        "size": 0,
        "query": {
        "bool": {
        "must": [
        {
            "range": {
            "@timestamp": {
                "gte": past,
                "lte": now,
                "format": "epoch_millis"
                 }
            }
        },
            {"term": {"event_type": "transfer-failed"}},
            {"term": {"payload.scope": "data16_13TeV"}}
        ]
        }
        }
    }


    
    # make scrolls
    #es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60) #chicago elasticsearch
    es = Elasticsearch(['es-atlas.cern.ch:9203'],timeout=60) #cern elasticsearch
    
    my_index = "rucio-events-2017*"
    scroll_done = scan(es, query=query_done, index=my_index, scroll='5m', timeout="5m", size=100)
    scroll_errors = scan(es, query=query_failed, index=my_index, scroll='5m', timeout="5m", size=100)
    
    for entry in scroll_done:
        transfers += 1
    
    for entry in scroll_errors:
        failures += 1
        
        
    if verbose: print('Number of successful transfers last ',interval,' seconds: ',transfers)
    if verbose: print('Number of failed transfers last ',interval,' seconds: ',failures)

    #schedule rerun of job in [interval] seconds
    print()
    s.enter(interval, 1, process_last, ())

In [82]:
process_last()
#s.run()


The clock is:  2017-04-25 03:47:03
Epoch time:  1493110023000


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f2e8bb35e48>: Failed to establish a new connection: [Errno 110] Connection timed out) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f2e8bb35e48>: Failed to establish a new connection: [Errno 110] Connection timed out)